In [1]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
import seaborn as sns

In [2]:
# Connect to database

connection_args = {
    'host': '18.221.53.115', # You'll have to update this to your IP
    'user': 'ubuntu',    # username
    'dbname': 'typology_17',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)
cursor = connection.cursor()

In [25]:
# Load into pandas
sql = 'SELECT * FROM mvp_set'
raw_df = pd.read_sql(sql, connection)
raw_df.head()

,qb27,income,relig,racecmb,educ2,sex,party,oftvote,age,qb28,qb29,qb31,qe1,qe2,qe3,qe11,qbx,q51ll,q51mm,q51pp
0,2.0,3.0,1.0,1.0,5.0,2.0,2.0,1.0,68.0,2.0,1.0,1.0,1.0,3.0,2.0,2.0,1.0,1.0,1.0,9.0
1,1.0,9.0,13.0,9.0,5.0,1.0,1.0,1.0,56.0,1.0,2.0,1.0,1.0,3.0,3.0,1.0,4.0,2.0,2.0,1.0
2,1.0,5.0,1.0,2.0,5.0,1.0,2.0,1.0,52.0,1.0,2.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,2.0,2.0
3,3.0,5.0,1.0,1.0,3.0,1.0,1.0,2.0,71.0,2.0,1.0,1.0,3.0,3.0,1.0,3.0,2.0,1.0,1.0,2.0
4,1.0,8.0,1.0,1.0,6.0,1.0,2.0,3.0,58.0,2.0,2.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,2.0


In [26]:
raw_df.shape

(2505, 20)

In [27]:
raw_df.isnull().sum()

qb27       0
income     0
relig      0
racecmb    0
educ2      0
sex        0
party      0
oftvote    0
age        0
qb28       0
qb29       0
qb31       0
qe1        0
qe2        0
qe3        0
qe11       0
qbx        0
q51ll      0
q51mm      0
q51pp      0
dtype: int64

In [28]:
raw_df.dtypes

qb27       float64
income     float64
relig      float64
racecmb    float64
educ2      float64
sex        float64
party      float64
oftvote    float64
age        float64
qb28       float64
qb29       float64
qb31       float64
qe1        float64
qe2        float64
qe3        float64
qe11       float64
qbx        float64
q51ll      float64
q51mm      float64
q51pp      float64
dtype: object

In [29]:
df = raw_df.astype(int)
df.head()

,qb27,income,relig,racecmb,educ2,sex,party,oftvote,age,qb28,qb29,qb31,qe1,qe2,qe3,qe11,qbx,q51ll,q51mm,q51pp
0,2,3,1,1,5,2,2,1,68,2,1,1,1,3,2,2,1,1,1,9
1,1,9,13,9,5,1,1,1,56,1,2,1,1,3,3,1,4,2,2,1
2,1,5,1,2,5,1,2,1,52,1,2,1,3,3,1,1,1,1,2,2
3,3,5,1,1,3,1,1,2,71,2,1,1,3,3,1,3,2,1,1,2
4,1,8,1,1,6,1,2,3,58,2,2,1,1,3,3,1,1,1,1,2


In [30]:
df.columns

Index(['qb27', 'income', 'relig', 'racecmb', 'educ2', 'sex', 'party',
       'oftvote', 'age', 'qb28', 'qb29', 'qb31', 'qe1', 'qe2', 'qe3', 'qe11',
       'qbx', 'q51ll', 'q51mm', 'q51pp'],
      dtype='object')

In [31]:
rename_dict = {
    'qb27' : 'more_engaged',
    'racecmb' : 'race',
    'educ2' : 'educ',
    'sex' : 'sex',
    'qb28' : 'int_con',
    'qb29' : 'local',
    'qb31' : 'glob_econ',
    'qbx' : 'econ_ineq',
    'qe2' : 'student',
    'qe1' : 'retired',
    'qe3' : 'employed',
    'qe11' : 'sav',
    'q51ll' : 'econ_unf',
    'q51mm' : 'pol_comp',
    'q51pp' : 'clear_sol' 
}

In [32]:
df = df.rename(rename_dict, axis=1)
df.head()

,more_engaged,income,relig,race,educ,sex,party,oftvote,age,int_con,local,glob_econ,retired,student,employed,sav,econ_ineq,econ_unf,pol_comp,clear_sol
0,2,3,1,1,5,2,2,1,68,2,1,1,1,3,2,2,1,1,1,9
1,1,9,13,9,5,1,1,1,56,1,2,1,1,3,3,1,4,2,2,1
2,1,5,1,2,5,1,2,1,52,1,2,1,3,3,1,1,1,1,2,2
3,3,5,1,1,3,1,1,2,71,2,1,1,3,3,1,3,2,1,1,2
4,1,8,1,1,6,1,2,3,58,2,2,1,1,3,3,1,1,1,1,2


In [11]:
total = df['more_engaged'].value_counts().sum()

for value in df['more_engaged'].unique():
    print('Response ' + str(value) + ' at ' + str(df['more_engaged'].value_counts()[value] / total ) + '%')

Response 2 at 0.11736526946107785%
Response 1 at 0.5377245508982036%
Response 3 at 0.33612774451097804%
Response 9 at 0.008782435129740519%


In [39]:
#Let's get some dummy variables!
dummify_list = ['more_engaged', 'int_con', 'local', 'glob_econ', 'econ_unf',
                'pol_comp', 'clear_sol', 'sex', 'race', 'relig', 'party']

In [40]:
df.columns

Index(['more_engaged', 'income', 'relig', 'race', 'educ', 'sex', 'party',
       'oftvote', 'age', 'int_con', 'local', 'glob_econ', 'retired', 'student',
       'employed', 'sav', 'econ_ineq', 'econ_unf', 'pol_comp', 'clear_sol'],
      dtype='object')

In [41]:
for column in dummify_list:
    dummies = pd.get_dummies(df[column], prefix = column, prefix_sep = '_')
    if column != 'relig':
        try:
            dummies = dummies.drop(f'{column}_9', axis=1)
        except:
            pass
        try:
            dummies = dummies.drop(f'{column}_10', axis = 1)
        except:
            pass
        try:
            dummies = dummies.drop(f'{column}_99', axis=1)
        except:
            pass
    else: 
        try:
            dummies = dummies.drop(f'{column}_99', axis=1)
        except:
            pass
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)

df.head()

,income,educ,oftvote,age,retired,student,employed,sav,econ_ineq,more_engaged_1,...,relig_10,relig_11,relig_12,relig_13,relig_14,party_1,party_2,party_3,party_4,party_5
0,3,5,1,68,1,3,2,2,1,0,...,0,0,0,0,0,0,1,0,0,0
1,9,5,1,56,1,3,3,1,4,1,...,0,0,0,1,0,1,0,0,0,0
2,5,5,1,52,3,3,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
3,5,3,2,71,3,3,1,3,2,0,...,0,0,0,0,0,1,0,0,0,0
4,8,6,3,58,1,3,3,1,1,1,...,0,0,0,0,0,0,1,0,0,0


In [42]:
df.columns

Index(['income', 'educ', 'oftvote', 'age', 'retired', 'student', 'employed',
       'sav', 'econ_ineq', 'more_engaged_1', 'more_engaged_2',
       'more_engaged_3', 'int_con_1', 'int_con_2', 'int_con_3', 'local_1',
       'local_2', 'local_3', 'glob_econ_1', 'glob_econ_2', 'glob_econ_3',
       'econ_unf_1', 'econ_unf_2', 'econ_unf_5', 'pol_comp_1', 'pol_comp_2',
       'pol_comp_5', 'clear_sol_1', 'clear_sol_2', 'clear_sol_5', 'sex_1',
       'sex_2', 'race_1', 'race_2', 'race_3', 'race_4', 'race_5', 'relig_1',
       'relig_2', 'relig_3', 'relig_4', 'relig_5', 'relig_6', 'relig_7',
       'relig_8', 'relig_9', 'relig_10', 'relig_11', 'relig_12', 'relig_13',
       'relig_14', 'party_1', 'party_2', 'party_3', 'party_4', 'party_5'],
      dtype='object')

In [48]:
# Give meaningful names to columns
relig_race_dict = {
    'relig_1' : 'protestant',
    'relig_2' : 'roman Catholic',
    'relig_3' : "mormon",
    'relig_4' : 'orth Chr',
    'relig_5' : 'jewish',
    'relig_6' : 'muslim',
    'relig_7' : 'buddhist',
    'relig_8' : 'hindu',
    'relig_9' : 'atheist',
    'relig_10' : 'agnostic',
    'relig_11' : 'relig_other',
    'relig_12' : 'relig_none',
    'relig_13' : 'christian',
    'relig_14' : 'unitarian',
    'race_1' : 'white',
    'race_2' : 'wlack',
    'race_3' : 'asian',
    'race_4' : 'race_other',
    'race_5' : 'native',
    'party_1' : 'republican',
    'party_2' : 'democrat',
    'party_3' : 'independent',
    'party_4' : 'party_none',
    'party_5' : 'party_other',
}

In [49]:
df = df.rename(relig_race_dict, axis=1)
df.head()

,income,educ,oftvote,age,retired,student,employed,sav,econ_ineq,more_engaged_1,...,agnostic,relig_Other,nothing,christian,unitarian,republican,democrat,independent,party_none,party_other
0,3,5,1,68,1,3,2,2,1,0,...,0,0,0,0,0,0,1,0,0,0
1,9,5,1,56,1,3,3,1,4,1,...,0,0,0,1,0,1,0,0,0,0
2,5,5,1,52,3,3,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
3,5,3,2,71,3,3,1,3,2,0,...,0,0,0,0,0,1,0,0,0,0
4,8,6,3,58,1,3,3,1,1,1,...,0,0,0,0,0,0,1,0,0,0


In [50]:
#Drop unneccessary dummies to simplify and clear out non-respondents

droplist = ['more_engaged_2', 'more_engaged_3', 'int_con_2', 'int_con_3', 'local_2',
            'local_3', 'glob_econ_2', 'glob_econ_3', 'econ_unf_2', 'econ_unf_5',
            'pol_comp_2', 'pol_comp_5', 'sex_2', 'clear_sol_2', 'clear_sol_5']

df = df.drop(droplist, axis=1)

KeyError: "labels ['more_engaged_2' 'more_engaged_3' 'int_con_2' 'int_con_3' 'local_2'\n 'local_3' 'glob_econ_2' 'glob_econ_3' 'econ_unf_2' 'econ_unf_5'\n 'pol_comp_2' 'pol_comp_5' 'sex_2' 'clear_sol_2' 'clear_sol_5'] not contained in axis"

In [51]:
df.columns

Index(['income', 'educ', 'oftvote', 'age', 'retired', 'student', 'employed',
       'sav', 'econ_ineq', 'more_engaged_1', 'int_con_1', 'local_1',
       'glob_econ_1', 'econ_unf_1', 'pol_comp_1', 'clear_sol_1', 'sex_1',
       'white', 'wlack', 'asian', 'race_other', 'native', 'protestant',
       'roman Catholic', 'mormon', 'orth Chr', 'jewish', 'muslim', 'buddhist',
       'hindu', 'atheist', 'agnostic', 'relig_Other', 'nothing', 'christian',
       'unitarian', 'republican', 'democrat', 'independent', 'party_none',
       'party_other'],
      dtype='object')

In [52]:
# Impute! Guess I'm cheating a bit here by doing it before train/test split -- I'll fix it if I have time

def generic_impute(x):
    if x == 9 or x == 6:
        return 2
    else:
        return x

def impute_oftvote(x):
    if x == 9 or x == 6:
        return 3
    else:
        return x

def impute_income(x):
    if x == 10:
        return 6
    else:
        return x

def impute_age(x):
    if x == 99:
        return 54
    else:
        return x

def impute_educ(x):
    if x == 9:
        return 5
    else:
        return x

def impute_retired(x):
    if x == 9 or x == 6 or x == 4:
        return 2
    else:
        return x

In [55]:
generic_clean = ['econ_ineq', 'student', 'employed', 'sav', 'retired']

for column in generic_clean:
    df[column] = df[column].apply(generic_impute)
    
df['oftvote'] = df['oftvote'].apply(impute_oftvote)
df['income'] = df['income'].apply(impute_income)
df['age'] = df['age'].apply(impute_age)
df['educ'] = df['educ'].apply(impute_educ)


In [54]:
for column in generic_clean:
    print(df[column].value_counts())

print(df.educ.value_counts())

1    1190
2     917
3     261
4     137
Name: econ_ineq, dtype: int64
3    2273
1     139
2      93
Name: student, dtype: int64
1    1203
3     938
2     364
Name: employed, dtype: int64
1    1086
3     944
2     475
Name: sav, dtype: int64
3    1598
1     820
2      49
4      38
Name: retired, dtype: int64
6    641
3    547
8    488
4    354
5    295
2     91
1     45
7     44
Name: educ, dtype: int64


In [56]:
df.head()

,income,educ,oftvote,age,retired,student,employed,sav,econ_ineq,more_engaged_1,...,agnostic,relig_Other,nothing,christian,unitarian,republican,democrat,independent,party_none,party_other
0,3,5,1,68,1,3,2,2,1,0,...,0,0,0,0,0,0,1,0,0,0
1,9,5,1,56,1,3,3,1,4,1,...,0,0,0,1,0,1,0,0,0,0
2,5,5,1,52,3,3,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
3,5,3,2,71,3,3,1,3,2,0,...,0,0,0,0,0,1,0,0,0,0
4,8,6,3,58,1,3,3,1,1,1,...,0,0,0,0,0,0,1,0,0,0


In [57]:
df = df.astype(int)
df.head()

,income,educ,oftvote,age,retired,student,employed,sav,econ_ineq,more_engaged_1,...,agnostic,relig_Other,nothing,christian,unitarian,republican,democrat,independent,party_none,party_other
0,3,5,1,68,1,3,2,2,1,0,...,0,0,0,0,0,0,1,0,0,0
1,9,5,1,56,1,3,3,1,4,1,...,0,0,0,1,0,1,0,0,0,0
2,5,5,1,52,3,3,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
3,5,3,2,71,3,3,1,3,2,0,...,0,0,0,0,0,1,0,0,0,0
4,8,6,3,58,1,3,3,1,1,1,...,0,0,0,0,0,0,1,0,0,0


In [58]:
df.to_pickle('03_mvp_pickle.pkl')